In [8]:
#| default_exp routes.sandbox

In [9]:
#| export
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [10]:
# | export


@gd.route_function
async def get_is_allow_same_instance_promotion_enabled(
    auth: dmda.DomoAuth,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop: int = 1,
    debug_api: bool = False,
    parent_class: str = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/version/v1/settings"

    res = await gd.get_data(auth=auth,
                         method='GET',
                         url=url,
                         session=session,
                         debug_api=debug_api,
                         num_stacks_to_drop = debug_num_stacks_to_drop,
                         parent_class = parent_class
                         )

    return res

In [11]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await get_is_allow_same_instance_promotion_enabled(auth = token_auth, debug_api = False)

ResponseGetData(status=200, response={'allowSelfPromotion': True, 'requireApprovals': False, 'domain': 'domo-community.domo.com'}, is_success=True, parent_class=None)

In [12]:
# | export


class Sandbox_ToggleSameInstancePromotion_Error(de.DomoError):
    def __init__(self, domo_instance, message, status, parent_class=None):
        super().__init__(
            self,
            domo_instance=domo_instance,
            status=status,
            parent_class=parent_class,
            message=message,
        )


@gd.route_function
async def toggle_allow_same_instance_promotion(
    is_enabled: bool,
    auth: dmda.DomoAuth,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop: int = 1,
    debug_api: bool = False,
    parent_class: str = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/version/v1/settings"

    body = {"allowSelfPromotion": is_enabled}

    res = await gd.get_data(
        auth=auth,
        method="POST",
        url=url,
        body=body,
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop = debug_num_stacks_to_drop
    )

    if not res.is_success:
        raise Sandbox_ToggleSameInstancePromotion_Error(
            domo_instance=auth.domo_instance,
            message=res.response,
            status=res.status,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

In [13]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await toggle_allow_same_instance_promotion(is_enabled = True, auth = token_auth, debug_api = False )

ResponseGetData(status=200, response={'allowSelfPromotion': True, 'requireApprovals': False, 'domain': 'domo-community.domo.com'}, is_success=True, parent_class=None)

In [14]:
#| export

@gd.route_function
async def get_shared_repos(auth: dmda.DomoAuth, 
                           session: httpx.AsyncClient = None,
                           parent_class :str = None,
                           debug_api: bool = False,
                           debug_num_stacks_to_drop : bool = False
                           ) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/version/v1/repositories/search"
    
    body = {
        "query": {
            "offset": 0,
            "limit": 50,
            "fieldSearchMap": {},
            "sort": "lastCommit",
            "order": "desc",
            "filters": {
                "userId": None
            },
            "dateFilters": {}
        },
        "shared": False
    }

    res = await gd.get_data(auth=auth,
                         method='POST',
                         url=url,
                         body=body,
                         session=session,
                         debug_api=debug_api,
                         parent_class = parent_class,
                         num_stacks_to_drop = debug_num_stacks_to_drop
                         )

    return res

Sample of getting shared repos

In [15]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


res = await get_shared_repos(auth=token_auth)

from pprint import pprint
pprint(res.response)

{'pageContext': {'count': 3, 'offSet': 0, 'totalCount': 3},
 'repositories': [{'accessCount': 1,
                   'created': '2023-06-15T21:09:00.244Z',
                   'domain': 'domo-community.domo.com',
                   'id': 'abd4cb2b-ec72-4373-a471-2a04db500713',
                   'lastCommit': {'commitName': 'v2',
                                  'completed': '2023-06-15T21:17:17.188Z',
                                  'hidden': False,
                                  'id': 'bc2b2d41-f1b8-4eb8-93f6-ecdf3074c00a',
                                  'pusherEventId': 'a900d3bb-595a-4f1e-b61f-5cbd9a19b841',
                                  'repositoryId': 'abd4cb2b-ec72-4373-a471-2a04db500713',
                                  'started': '2023-06-15T21:17:16.334Z',
                                  'status': 'COMPLETED',
                                  'summary': 'uat accepted jira 1234'},
                   'name': 'Landing Page_PROD',
                   'permission': 

In [16]:
# | export
@gd.route_function
async def get_repo_from_id(auth: dmda.DomoFullAuth,
                           repository_id: str,
                           debug_api: bool = False,
                           debug_num_stacks_to_drop : int = 1,
                           parent_class :str = None,
                           session: httpx.AsyncClient = None
                           ) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/version/v1/repositories/{repository_id}"

    return await gd.get_data(auth=auth,
                          method='GET',
                          url=url,
                          parent_class = parent_class,
                          debug_api=debug_api,
                          num_stacks_to_drop = debug_num_stacks_to_drop
                          )


In [17]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


res = await get_shared_repos(auth=token_auth)

res = await get_repo_from_id(
    auth=token_auth, repository_id=res.response["repositories"][0]["id"]
)

res.response

{'id': 'abd4cb2b-ec72-4373-a471-2a04db500713',
 'domain': 'domo-community.domo.com',
 'name': 'Landing Page_PROD',
 'accessCount': 1,
 'type': 'dataflow',
 'repositoryContent': {'pageIds': [],
  'viewIds': [],
  'dataflowIds': [226],
  'streamIds': [],
  'cardIds': []},
 'userId': 1893952720,
 'created': '2023-06-15T21:09:00.244Z',
 'updated': '2023-06-15T21:09:00.244Z',
 'seeded': False,
 'lastCommit': {'id': 'bc2b2d41-f1b8-4eb8-93f6-ecdf3074c00a',
  'repositoryId': 'abd4cb2b-ec72-4373-a471-2a04db500713',
  'commitName': 'v2',
  'hidden': False,
  'summary': 'uat accepted jira 1234',
  'started': '2023-06-15T21:17:16.334Z',
  'completed': '2023-06-15T21:17:17.188Z',
  'status': 'COMPLETED',
  'pusherEventId': 'a900d3bb-595a-4f1e-b61f-5cbd9a19b841'},
 'deployments': [{'id': 'fda423b7-c53a-4426-a52d-77f8d9a3ff4b',
   'repositoryId': 'abd4cb2b-ec72-4373-a471-2a04db500713',
   'domain': 'domo-community.domo.com',
   'name': 'Landing Page_PROD',
   'isSource': False,
   'userId': 189395272

In [18]:
#| hide
import nbdev

nbdev.nbdev_export()